# Lab Optimización - Programación Lineal

Dentro del campo de la análitica avanzada existe lo que se denomina Operations Research (Investigación operativa), esta se encarga de aplicar métodos de optimización para la toma de decisiones de negocio. 

Cuando hablamos de optimización, nos referimos a máximizar o mímimizar alguna función, esta puede representar costos, tiempo, revenue, ventas, producción, etc. Adicionalmente a esa función necesitmos expresar en el modelo todas las reglas que le dan un sentido de negocio al problema.

### Programación Líneal

La principal herramienta en está área es la programación líneal (PL), esto es, usar funciones líneales para traducir de forma mátematica el problema a optimizar.

Para esto necesitamos modelar 3 cosas:

* Variables del problema: Generalmente representan las decisiones a tomar. 
* Función objetivo: Lo que queremos optimizar, sea para máximizar o mínimizar. 
* Restricciones: Definidas por desigualdades cerradas, representan las reglas de negocio que se deben cumplir.

Es importante tomar en cuenta las unidades de las variables, así como sus magnitudes a la hora de modelar las mismas, así como también tomar en cuenta que las funciones en el modelo deben de ser lineales, es decir, no se pueden multiplicar variables ni aplicar exponenciales, logaritmos, etc. Aunque si se pueden realizar aproximaciones para dichas funciones (Ej: Envoltura de McCormik para una función cuadratica).

Un modelo de PL tiene un aspecto similar a:

$$max \sum_{i=1}^{n} X_{i}$$
$$Sub to:$$
$$\sum a_{i} X_{i} \leq M$$
$$4 X_{1} - 2 X_{2} \leq 10$$

#### Ejemplo:

Andreani tiene $M$ sucursales que reparten en la ciudad de Buenos Aires, cada sucursal tiene una capacidad máxima $C_{s}$ de envíos diarios que puede procesar, además, el costo de procesamiento por envio $O_{s}$ de cada sucursal es diferente. 

Queremos distribuir $N$ envíos entre las 3 sucursales a costo mínimo, sin embargo, ninguna sucursal puede procesar menos de $\frac{3}{4}$ que ninguna otra sucursal por temas contractuales.

##### **Definición del problema de PL**

**Variables:**

Sea $X_{s}$ la cantidad de envíos que procesa la sucursal $s$.

**Función Objetivo:**

Queremos minimizar el costo, luego la función objetivo es la suma de los costos de cada sucursal.

$$min \sum_{s=1}^{M} O_{s}  X_{s}$$

**Restricciones:**

Ninguna sucursal puede procesar una cantidad de envíos mayor a su capacidad.

$$X_{s} \leq C_{s} \forall s \in S$$

Ninguna sucursal puede procesar menos de 3/4 de otra sucursal

$$\frac{3 X_{i}}{4} \leq X_{s} \forall s, i \in S : s \neq i$$

Debemos entregar todos los envios

$$\sum_{s=1}^{M} X_{s} = N$$

Nota: S es el conjunto de todas las sucursales

##### **Solución**

Para resolver un PL se tiene una gran variedad de _solvers_ algunos gratuitos y otros pagos, en este caso usaremos la libreria [ORTools](https://pypi.org/project/ortools/) de Google la cual nos permite usar [SCIP](https://www.scipopt.org/).

In [ ]:
%%time
import pandas as pd
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
data = pd.read_csv("data/branches_example.csv", index_col = "branch_name")
N = 5000

# Declaramos las variables.
variables = {f"X_{branch}" : solver.NumVar(0, N, f"X_{branch}") for branch in data.index}

# Declaramos la función objetivo
solver.Minimize(sum(branch.cost * variables[f"X_{branch_name}"] for branch_name, branch in data.iterrows()))

# Declaramos las restricciones
## Capacidad
for var_name, var in variables.items():
    solver.Add(var <= data.capacity[var_name.split("_")[1]])
## Contrato
for var_name_s, var_s in variables.items():
    for var_name_i, var_i in variables.items():
        if var_name_s == var_name_i:
            continue
        solver.Add(3 * var_i / 4 <= var_s)
## Envios
solver.Add(sum(var for var_name, var in variables.items()) == N)

# Resolvemos
status = solver.Solve()

# Resulados
if status == pywraplp.Solver.OPTIMAL:
    print(f"Costo Total: {solver.Objective().Value()}")
    print("Envios por sucursal:")
    for var_name, var in variables.items():
        print(f"{var_name.split('_')[1]}: {var.solution_value()}")
else:
    print("No se encontro el optimo")

### Programación Líneal Entera

En el ejemplo anterior vemos que el resultado no es un número entero, y, para la mayoria de problemas, así como el del ejemplo, no sirven los resultados con respuestas no enteras, sin embargo, el costo computacional para resolver un problema de programación lineal con variables enteras se incrementa debido a que no existen algoritmos de resolución optimos (En general son de orden polinomico e incluso exponencial), por lo que modelar los problemas de una forma más sencilla cobra mayor imporancia.

En el ejemplo anterior solo debemos cambiar la definición de las variables

#### Ejemplo

In [ ]:
%%time
solver = pywraplp.Solver.CreateSolver('SCIP')
data = pd.read_csv("data/branches_example.csv", index_col = "branch_name")
N = 5000

# Declaramos las variables.
variables = {f"X_{branch}" : solver.IntVar(0, N, f"X_{branch}") for branch in data.index}

# Declaramos la función objetivo
solver.Minimize(sum(branch.cost * variables[f"X_{branch_name}"] for branch_name, branch in data.iterrows()))

# Declaramos las restricciones
## Capacidad
for var_name, var in variables.items():
    solver.Add(var <= data.capacity[var_name.split("_")[1]])
## Contrato
for var_name_s, var_s in variables.items():
    for var_name_i, var_i in variables.items():
        if var_name_s == var_name_i:
            continue
        solver.Add(3 * var_i / 4 <= var_s)
## Envios
solver.Add(sum(var for var_name, var in variables.items()) == N)

# Resolvemos
status = solver.Solve()

# Resulados
if status == pywraplp.Solver.OPTIMAL:
    print(f"Costo Total: {solver.Objective().Value()}")
    print("Envios por sucursal:")
    for var_name, var in variables.items():
        print(f"{var_name.split('_')[1]}: {var.solution_value()}")
else:
    print("No se encontro el optimo")

Un caso particularmente util de la PLE son las variables binarias, las cuales no permiten modelar una gran variadad de casos que con variables reales seria imposible, entre estos casos estan:

* Asignación de tareas.
* Costos fijos por uso. 
* Costos escalonados. 
* Valores mínimos de uso (si se usa).

#### Ejemplo

En el mismo ejemplo anterior, se cambian las conidciones del contrato con cada sucursal, y se define que, cada sucursal tiene un costo de apertura $A_{s}$ el cual solo se gnera si la sucursal es usada y no depende de la cantidad de envíos que procese la sucursal, adicionalmente, para que cada sucursal sea usada debe de procesar al menos $L_{s}$ envíos.

##### **Definición del problema de PLE**

**Variables:**

Sea $X_{s}$ (entera) la cantidad de envíos que procesa la sucursal $s$.

Sea $Y_{s}$ (binaria) una indicadora de si la sucursal $s$ fue usada o no.

**Función Objetivo:**

Queremos minimizar el costo, luego la función objetivo es la suma de los costos de cada sucursal.

$$min \sum_{s=1}^{M} (O_{s}  X_{s} + A_{s}  Y_{s})$$

**Restricciones:**

Ninguna sucursal puede procesar una cantidad de envíos mayor a su capacidad.

$$X_{s} \leq C_{s} Y_{s} \forall s \in S$$

Debemos entregar todos los envios

$$\sum_{s=1}^{M} X_{s} = N$$

Si una sucursal es usada debe procesar al menos $L_{s}$ envíos.

$$L_{s} Y_{s} \leq X_{s} \forall s \in S$$

##### **Solución**

In [ ]:
%%time
solver = pywraplp.Solver.CreateSolver('SCIP')
data = pd.read_csv("data/branches_example_2.csv", index_col = "branch_name")
N = 5000

# Declaramos las variables.
shipments = {f"X_{branch}" : solver.IntVar(0, N, f"X_{branch}") for branch in data.index}

branches = {f"Y_{branch}" : solver.BoolVar(f"Y_{branch}") for branch in data.index}

# Declaramos la función objetivo
solver.Minimize(sum(branch.cost * shipments[f"X_{branch_name}"] + branch.open_cost * branches[f"Y_{branch_name}"]for branch_name, branch in data.iterrows()))

# Declaramos las restricciones
## Capacidad
for branch_name, branch in data.iterrows():
    solver.Add(shipments[f"X_{branch_name}"] <= branch.capacity * branches[f"Y_{branch_name}"])
## Envios
solver.Add(sum(var for var_name, var in shipments.items()) == N)
## Contrato
for branch_name, branch in data.iterrows():
    solver.Add(branch.min_shipments * branches[f"Y_{branch_name}"] <= shipments[f"X_{branch_name}"])

# Resolvemos
status = solver.Solve()

# Resulados
if status == pywraplp.Solver.OPTIMAL:
    print(f"Costo Total: {solver.Objective().Value()}")
    print("Envios por sucursal:")
    for var_name, var in variables.items():
        print(f"{var_name.split('_')[1]}: {var.solution_value()}")
else:
    print("No se encontro el optimo")